In [200]:
import numpy as np
import numpy as np
import commpy as cp
import scipy.signal as sig
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
"""# EXAMPLE DATA GENERATION SEQUENCE
# define the number of constellation points
c=4 
# length of array of bits (must be divisible by c)
n=4*c

# create the modulation object with c constellation points
QAMModem = cp.modulation.QAMModem(c)
# create a random array of bits
input_data_bits=np.random.randint(0,2,n) 
# use the constellation object to modulate the data bits into complex numbers
input_data_constellations = QAMModem.modulate(input_data_bits)
print(input_data_bits)
print(input_data_constellations)
# an example of a channel function with two consecutive taps
channel_function=[0.7,0.3]
# convolution of the input complex data with the channel transfer function
channel_output = sig.convolve(input_data_constellations, channel_function, mode='full')
print(channel_output)

a = zfEqualize(channel_output, channel_function)
print(len(channel_output))
print(len(channel_function))
print(len(a))
for i in range(len(input_data_constellations)):
    print(input_data_constellations[i], "    ", a[i])
    print(np.abs(input_data_constellations[i] - a[i]) < 5e-10)

#print(input_data_constellations - a < 5e-10)
print(QAMModem.demodulate(a, 'hard'))
print(numFalses(input_data_bits, a, QAMModem))"""

In [206]:
# Function to zero force equalize. 
def zfEqualize(channel_output, channel):
    # make sure have same fft length 
    # we get the correct number of terms as this is like a circular convolution so the last packet is garbage
    freq_domain = np.fft.fft(channel_output, len(channel_output))/np.fft.fft(channel, len(channel_output))
    return np.fft.ifft(freq_domain)[0:len(channel_output) - len(channel) + 1]

# Count number of symbols that are off
def numFalses(inbits, equalized_constellation, QAMModem):
    outbits = QAMModem.demodulate(equalized_constellation, "hard")
    total = 0
    for i in range(len(inbits)):
        if inbits[i] != outbits[i]:
            total += 1
    return total

# Adds complex gaussian noise to inputted signal 
def addNoise(input_signal, noise_amp):   
    n = len(input_signal)
    noise = noise_amp*np.random.randn(2*n)
    noisy_array = []
    for i in range(len(input_signal)):
        # used to construct complex noise with gaussian rv on both real and imag terms
        noisy_array.append(input_signal[i] + noise[i] + noise[i+1]*1j)
    return noisy_array

# Generate data samples run them through a channel then equalize using zero force equalizer and return symbol error rate
def find_SER(channel_len, noise_amp, runs = 1000, num_bits = 4 ):
    # make modulator and message size
    QAMModem = cp.modulation.QAMModem(c)
    constellation = 4
    n = constellation * num_bits
    sym_errors = 0 

    for i in range(runs):
        # generate input bits and constellation
        input_bits = np.random.randint(0,2,n) 
        input_constellation = QAMModem.modulate(input_bits)
        # pass signal through random channel
        channel_function= np.random.randn(channel_len)
        channel_output = sig.convolve(input_constellation, channel_function, mode="full")
        channel_output = addNoise(channel_output, noise_amp)
        # equalize, demodulate and calculate the symbol error rate
        eqd = zfEqualize(channel_output, channel_function)
        falses = numFalses(input_bits, eqd, QAMModem)
        sym_errors += falses
    sym_error_rate = sym_errors/(n*runs)
    return sym_error_rate

In [ ]:
# Calculate the symbol error rate for different noise magnitudes
noise_mag_errs = []
for i in np.arange(0, 1, 0.02):
    noise_mag_errs.append(find_SER(2, i))

In [ ]:
# Calculate the symbol error rate for different channel lengths for fixed noise magnitude = 0.4
channel_mag_errs = []
for i in np.arange(2, 20):
    channel_mag_errs.append(find_SER(i, 0.4))

In [ ]:
# Plot symbol error rate vs. noise magnitude and symbol error rate vs. channel length
print(noise_mag_errs, "\n")
plt.figure()
plt.title("Symbol error rate vs. Noise magnitude for fixed Channel Length = 2")
plt.plot(np.arange(0, 1, 0.02), noise_mag_errs)

print("\n", channel_mag_errs, "\n")
plt.figure()
plt.title("Symbol error rate vs. Channel Length for fixed Noise magnitude = 0.4")
plt.plot(np.arange(2,20), channel_mag_errs)